# Type Hierarchy

Let's look at the types `PolyChaos` provides.
The high-level perspective looks as such:
![type hierarchy](TypeHierarchy_transparent.png)

which we will go through step by step.
An arrow beginning at one type and ending at another type means that the beginning type is a field of the ending type.
For example, the type `OrthoPoly` has a field of type `Measure`; the type `OrthoPolyQ` has a field of type `OrthoPoly` and a field of type `Quad`, and so on.

Let's begin with the univariate case. Our running example will be the Gaussian measure.

!!! note
    If you are unfamiliar with the mathematical background of orthogonal polynomials, please consult [this tutorial](@ref MathematicalBackground).
    

## Measure
It all begins with a measure, more specifically absolutely continuous measures.
What are the fields of such a type `measure`?
```
name::String
w::Function
dom::Tuple{Float64,Float64}
symmetric::Bool
pars::Dict
```
They are a `name`, a weight function $w: \Omega \mapsto \mathbb{R}$ with domain $\Omega$ (`dom`).
If the weight function is symmetric relative to some $m \in \Omega$, the field `symmetric` should be set to `true`.
Symmetry relative to $m$ means that
$$
\forall x \in \Omega: \quad w(m-x) = w(m+x).
$$
For example, the Gaussian probability density
$$
w(x) = \frac{1}{\sqrt{2\pi}} \mathrm{e}^{-x^2/2}
$$
is symmetric relative to the origin $m=0$.
If the weight function has any parameters, then they are stored in the dictionary `pars`.
For example, the probability density of the Beta distribution on $\Omega = [0,1]$ has two positive shape parameters $\alpha, \beta > 0$
$$
w(x) = \frac{1}{B(\alpha,\beta)} x^{\alpha-1} (1-x)^{\beta-1}.
$$
Several measures come built-in with `PolyChaos`(see [here](@ref CommonRandomVariables)).
Let's stick to the Gaussian measure:


In [16]:
using Revise
using PolyChaos
m_gauss = Measure("gaussian")

Measure dλ(t)=w(t)dt
name:		gaussian
w(t):		PolyChaos.w_gaussian
dom:		(-Inf, Inf)
symmetric:	true


The manual way of defining the same thing (and what `PolyChaos` does under the hood for this example) would be

In [17]:
name = "mygaussian"
w(x) = 1/sqrt(2*pi)*exp(-0.5*x^2)
dom = (-Inf,Inf)
symmetric = true
pars = Dict()
my_m_gauss = Measure(name,w,dom,symmetric,pars)

Measure dλ(t)=w(t)dt
name:		mygaussian
w(t):		w
dom:		(-Inf, Inf)
symmetric:	true


## OrthoPoly
Given an absolutely continuous measure we are wondering what are the monic polynomials $\phi_i: \Omega \rightarrow \mathbb{R}$ that are orthogonal relative to this very measure?
Mathematically this reads
$$
\langle \phi_i, \phi_j \rangle = \int_{\Omega} \phi_i(t) \phi_j(t) w(t) \mathrm{d}t =
\begin{cases}
> 0, & i=j \\
= 0, & i\neq j.
\end{cases}
$$
They can be constructed using the type `OrthoPoly`, which has the fields
```
name::String
deg::Int64          # maximum degree
α::Vector{Float64}  # recurrence coefficients
β::Vector{Float64}  # recurrence coefficients
meas::Measure
```
The purpose of `name` is obvious.
The integer `deg` stands for the maxium degree of the polynomials.
Rather than storing the polynomials $\phi_i$ themselves we store the recurrence coefficients `α`, `β` that characterize the system of orthogonal polynomials.
These recurrence coefficients are the single most important piece of information for the orthogonal polynomials.
For several common measures, there exist analytic formulae.
These are built-in to `PolyChaos` and should be used whenever possible.
For example, for a Gaussian probability density we can call:

In [37]:
d = 6
op = OrthoPoly("gaussian",d)


Univariate orthogonal polynomials
name:		gaussian
degree:		6
#coeffs:	7
α =		[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
β =		[1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

Measure dλ(t)=w(t)dt
name:		gaussian
w(t):		PolyChaos.w_gaussian
dom:		(-Inf, Inf)
symmetric:	true


On the other hand, `PolyChaos` allows to construct the recurrence coefficients numerically.
Possible discretizations are `stieltjes` and `lanczos`; the underlying quadratures can be either one of `clenshaw_curtis`, `fejer`, or `fejer2`.

In [38]:
my_op = OrthoPoly("mygaussian",d,my_m_gauss;Nquad=200,quadrature=clenshaw_curtis,discretization=lanczos)


Univariate orthogonal polynomials
name:		mygaussian
degree:		6
#coeffs:	7
α =		[-9.72317e-17, 2.83503e-16, -9.41033e-16, 1.09066e-15, 1.03878e-15, -2.0357e-15, -6.08671e-16]
β =		[1.0, 1.0, 2.0, 3.0, 4.0, 4.99999, 5.99995]

Measure dλ(t)=w(t)dt
name:		mygaussian
w(t):		w
dom:		(-Inf, Inf)
symmetric:	true


Let's compare the values of the recurrence coefficients

In [39]:
coeffs(op) - coeffs(my_op)

7×2 Array{Float64,2}:
  9.72317e-17  3.59268e-13
 -2.83503e-16  1.98234e-11
  9.41033e-16  1.0521e-9  
 -1.09066e-15  2.68327e-8 
 -1.03878e-15  4.37355e-7 
  2.0357e-15   5.11028e-6 
  6.08671e-16  4.55036e-5 

Having constructed the orthogonal polynomials we can evaluate them with `evaluate`.
For example, let's evaluate all polynomials from degree $0...d$ at $x=3.$

In [40]:
[ evaluate(d_,3.,op) for d_=1:d ]

6-element Array{Float64,1}:
   3.0
   8.0
  18.0
  30.0
  18.0
 -96.0

## Quad
Quadrature rules are intricately related to orthogonal polynomials.
An $n$-point quadrature rule is a pair of so-called nodes $t_k$ and weights $w_k$ for $k=1,\dots,n$ that allow to solve integrals relative to the measure
$$
\int_\Omega f(t) w(t) \mathrm{d} t \approx \sum_{k=1}^n w_k f(t_k).
$$
If the integrand $f$ is polynomial, then the specific Gauss quadrature rules possess the remarkable property that an $n$-point quadrature rule can integrate polynomial integrands $f$ of degree at most $2n-1$ *exactly*; no approximation error is made.

For common measures, `PolyChaos` resorts to the package [`FastGaussQuadrature`](https://github.com/ajt60gaibb/FastGaussQuadrature.jl/)

!!! note
    The compilation time of `FastGaussQuadrature` is currently extremely slow, [see here](https://github.com/ajt60gaibb/FastGaussQuadrature.jl/issues/47).

In [41]:
n=5
q = Quad("gaussian",n)


Quadrature rule
name:		gaussian
N:		5
nodes		[-2.85697, -1.35563, 1.33227e-15, 1.35563, 2.85697]
weights		[0.0112574, 0.222076, 0.533333, 0.222076, 0.0112574]

Measure dλ(t)=w(t)dt
name:		gaussian
w(t):		PolyChaos.w_gaussian
dom:		(-Inf, Inf)
symmetric:	true


`PolyChaos` allows to compute the quadrature rule for arbitrary measures based on the orthogonal polynomials by using the Golub-Welsch-algorithm.
This is done as follows

In [42]:
my_q = Quad(n,op)


Quadrature rule
name:		golubwelsch
N:		5
nodes		[-2.85697, -1.35563, 1.33227e-15, 1.35563, 2.85697]
weights		[0.0112574, 0.222076, 0.533333, 0.222076, 0.0112574]

Measure dλ(t)=w(t)dt
name:		gaussian
w(t):		PolyChaos.w_gaussian
dom:		(-Inf, Inf)
symmetric:	true


Again, let's compare the results

In [43]:
nw(q) - nw(my_q)

5×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

!!! note
    Once more---[for common measures](@ref CommonRandomVariables), `PolyChaos` has built-in functionality, such as analytic expressions of the recurrence coefficients.
    These should always be used, if applicable.
    On the other hand, `PolyChaos` allows to construct orthogonal polynomials and quadrature rules for arbitrary weights.
    
## OrthoPolyQ
As you would expect from the figure at the top, the type `OrthoPolyQ` is an amalgamation of `OrthoPoly` and `Quad`.
It has just those two fields
```
op::OrthoPoly
quad::Quad
```
The construction is straightforward

In [45]:
opq = OrthoPolyQ(op,q)
myopq = OrthoPolyQ(my_op,my_q)


Univariate orthogonal polynomials
name:		mygaussian
degree:		6
#coeffs:	7
α =		[-9.72317e-17, 2.83503e-16, -9.41033e-16, 1.09066e-15, 1.03878e-15, -2.0357e-15, -6.08671e-16]
β =		[1.0, 1.0, 2.0, 3.0, 4.0, 4.99999, 5.99995]

Measure dλ(t)=w(t)dt
name:		mygaussian
w(t):		w
dom:		(-Inf, Inf)
symmetric:	true

Quadrature rule
name:		golubwelsch
N:		5
nodes		[-2.85697, -1.35563, 1.33227e-15, 1.35563, 2.85697]
weights		[0.0112574, 0.222076, 0.533333, 0.222076, 0.0112574]


There is an even simpler constructor, which constructs `OrthoPolyQ` in a single shot

In [44]:
opq = OrthoPolyQ("gaussian",d)


Univariate orthogonal polynomials
name:		gaussian
degree:		6
#coeffs:	7
α =		[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
β =		[1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

Measure dλ(t)=w(t)dt
name:		gaussian
w(t):		PolyChaos.w_gaussian
dom:		(-Inf, Inf)
symmetric:	true

Quadrature rule
name:		gaussian
N:		7
nodes		[-3.75044, -2.36676, -1.15441, 2.66454e-15, 1.15441, 2.36676, 3.75044]
weights		[0.000548269, 0.0307571, 0.240123, 0.457143, 0.240123, 0.0307571, 0.000548269]


## MultiMeasure
So far, everything was univariate, the weight of the measure was mapping real numbers to real numbers.
`PolyChaos` can handle product measures too.
Let's assume the weight function is a product of two independent Gaussian densities
$$
w: \mathbb{R} \times \mathbb{R} \rightarrow \mathbb{R}, \quad w(x) = \frac{1}{\sqrt{2\pi}} \mathrm{e}^{x_1^2/2} \frac{1}{\sqrt{2\pi}} \mathrm{e}^{x_2^2/2}.
$$
The type `MultiMeasure` serves this purpose, with its fields
```
name::Vector{String}
w::Function
w_uni::Vector{Function}
dom::Vector{Tuple{Float64,Float64}}
symmetric::Vector{Bool}
pars::Vector{Dict}
```
All fields from `Measure` appear in vectorized versions (except for the weight $w$, which is the weight of the product measure)
The only *new* field is `w_uni`, which stacks the univariate weight functions.

## MultiOrthoPoly
Just as we did in the univariate case, we use `MultiMeasure` as a building block for multivariate orthogonal polynomials.
The type `MultiOrthoPoly` combines product measures with the respective orthogonal polynomials and their quadrature rules.
Its fields are
```
name::Vector{String}
deg::Int64
dim::Int64
ind::Matrix{Int64} # multi-index
meas::MultiMeasure
uni::Union{Vector{OrthoPoly},Vector{OrthoPolyQ}}
```
The names of the univariate bases are stored in `names`; the maximum degree of the basis is `deg`; the overall dimension of the multivariate basis is `dim`; the multi-index `ind` maps the $j$-th multivariate basis to the elements of the univariate bases; the product measure is stored in `meas`; finally, all univariate bases are collected in `uni`.

An example will shed light onto the different notions:

In [47]:
maxd = 4
mop = MultiOrthoPoly([opq; opq; opq],maxd)


3-variate orthogonal polynomials
name:		gaussian
		gaussian
		gaussian
deg:		4
dim:		35
ind:		[0, 0, 0]
		[1, 0, 0]
		[0, 1, 0]
		[0, 0, 1]
		[2, 0, 0]
		[1, 1, 0]
		[1, 0, 1]
		...



This creates a product measure of three univariate measures.
The maximum degree is 4, which leads to an overall basis dimension of 35.
For a more detailed discussion please [click here](MultivariateMonicOrthogonalPolynomials).

## Tensor
The last type we need to address is `Tensor`.
It is used to store the results of scalar products, [see here](ComputationOfScalarProducts).
Its fields are
```
dim::Int64          # "dimension"
T::SparseVector{Float64,Int64}
get::Function
op::Union{OrthoPolyQ,MultiOrthoPoly}
```